In [1]:
import pandas as pd
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import LlamaCpp
from tqdm import tqdm

In [2]:
df = pd.read_csv('./data_merge.csv')

In [5]:
df['part_3'] = [x.split('III')[-1] for x in df['text']]

In [ ]:
llm = LlamaCpp(
    model_path="../models/Mixtral-8x7B-Instruct-v0.1-GGUF/mixtral-8x7b-instruct-v0.1.Q6_K.gguf",
    n_gpu_layers=50,
    n_batch=512,
    n_ctx=3500,
    f16_kv=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True,
    temperature = 0
)

In [20]:
schema = Object(
    id="application_data",
    description="Tender application data",
    attributes=[
        Text(
          id='basic_qualification',
          description="Basic candidate qualifications",
          examples=[("""участник процедур, зарегистрированный на территории Китайской Народной Республики в соответствии с Законом Китайской Народной Республики о компаниях, имеет право
            Законодательная власть, предприятия, которые могут предложить продукты и услуги для проекта.""",
                    """участник процедур должен быть зарегистрирован на территории Китайской Народной Республики в соответствии с Законом Китайской Народной Республики о компаниях
            Независимые юридические лица, предприятия, способные предлагать продукты и услуги для проекта.""")],
            ),
        Text(
          id='qualification_requirements',
          description="Qualification requirements",
          examples=[("""Генеральный контрактный уровень (или уровень А) и выше в области электротехники.""",
                    """Для участия в торгах требуется, чтобы участники торгов обладали такими качествами, как общий контракт на строительство гидроэлектроэнергии и выше. (если в соответствии с
Министерство по вопросам городского и сельского строительства в связи с выпуском циркуляра о программе реформы системы управления инженерно-строительными предприятиями в целях повышения квалификации или новых сертификатов для гидроэлектроэнергетики
Генеральный контракт на строительство уровня А и выше).""")],
            ),
        Text(
          id='financial_requirements',
          description="Financial requirements",
          examples=[("""Финансовые требования: аудиторские заключения и финансовые ведомости, представленные зарегистрированной аудиторской фирмой за последние три года (2020-2022 годы)
балансовая ведомость, ведомость прибылей, ведомость движения денежной наличности и приложение.""",
                    """Отчет о финансовой ревизии за последние три года (2019-2021).""")],
            ),
        Text(
          id='performance_requirements',
          description="Performance requirements",
          examples=[("""должны иметь историю и результативность как минимум на три года производства товаров или аналогичных товаров. (Оборудование для замены коробки должно быть
            Комбинированная или сборная с емкостью 3150 КВА и выше, электрическое давление на 35 КВ и выше.""",
                    """в течение почти трех лет (с момента подачи тендерной документации, по крайней мере, одного крана и поставки
Установка работы.""")],
            ),
        Text(
          id='credit_requirements',
          description="Credit requirements",
          examples=[(""" участники торгов имеют хорошую банковскую и коммерческую репутацию, не находятся в ситуации, когда им предписывается прекратить работу, захват имущества, замораживание
            Конкретно, несостоятельно и не находится в пределах и в течение срока, в течение которого компания &lt; &lt; Чайна холдинг компани лимитед &gt; &gt; ограничила свои предложения.""",
                    """не включен в систему раскрытия информации о государственных предприятиях (www.gsxt.gov.cn)
                      В список и на веб-сайте «Доверие в Китай» (http://www.creditchina.gov.cn/) не было внесено в список «Посланник с неверием».""")],
            ),
        Text(
          id='tender_start_date',
          description="Start date for receiving tender documentation",
          examples=[("""07 июня 2023 года, 17:00""",
                    """28 декабря 2022 года в 17 ч. 00 м.""")],
            ),
        Text(
          id='tender_end_date',
          description="End date for receiving tender documentation",
          examples=[("""16 июня 2023 года, 17:00""",
                    """06 января 2023 года 17 ч. 30 м.""")],
            ),
        Text(
          id='trust_requirements',
          description="Requirement of trust",
          examples=[("""не находится в пределах и в течение срока, в течение которого компания &quot; Чайна Хинчжи &quot; ограничивает свои предложения.""",
                    """Компания &lt; &lt; Чайна-Групп &gt; &gt; не находится в ограниченном диапазоне и сроках проведения торгов.""")],
            ),  
    ],
    many=False,
)


In [21]:
chain = create_extraction_chain(llm, schema,encoder_or_encoder_class='json')

In [23]:
result = []
for i_text in tqdm(df['part_3']):
    try:
        ans = chain.run(text=(i_text))
        result.append(ans['data'])
    except:
        result.append({'basic_qualification': '', 'qualification_requirements': '',
   'financial_requirements': '', 'performance_requirements': '', 'credit_requirements': '', 
   'tender_start_date': '',
   'tender_end_date': '',})

100%|██████████| 446/446 [23:29<00:00,  3.16s/it]


In [31]:
temp_list = []
for i in tqdm(range(len(result))):
    if len(result[i]) > 0:
        temp_list.append(result[i]['application_data'])

100%|██████████| 162/162 [00:00<00:00, 1496645.92it/s]


In [32]:
temp_df = pd.DataFrame(temp_list)

In [34]:
temp_df.to_csv('./data/part3_for_merge_PART.csv', index=False)